### This notebook uses a breathing actuation sequence that computes triggers inflation deflation based upon an average inhale/exhale duration and a scale factor

In [8]:
from pythonosc import dispatcher, osc_server
from pythonosc.udp_client import SimpleUDPClient
import biofeatures
import threading
import numpy as np

In [206]:
def data_handler(unused_addr, args, data1, data2, data3, data4, data5, data6): #BITalino ServerBIT format
    global ecg_data, resp_data
    global osc_client
        
    B = args[0]
    HR = args[1]
    
    # BITalino ServerBIT
    ecg_data.append(float(np.random.rand(1)))
    resp_data.append(float(data6))
        
    if B.is_warmed_up:
        B.set_data(resp_data[-B.buffer_length:])
    
    if HR.is_warmed_up:
        HR.set_data(ecg_data[-HR.buffer_length:])


In [207]:
def warmup(B, resp_data, HR, ecg_data):
    """Function to launch once the system has warmed up.
    Sets the  data initially and launches a recursive update of features.
    Parameters
    ----------
    B: object containing breathing data and features
    resp_data: respiration signal data to set
    HR: object containing ECG data and features
    ecg_data: ecg signal data to set
    """
    
    if not B.is_warmed_up:
        print("Breathing WARMUP")
        B.set_data(resp_data[-B.buffer_length:])
        B.is_warmed_up = True
        B.update_loop()
        
    if not HR.is_warmed_up:
        print("ECG WARMUP")
        HR.set_data(ecg_data[-HR.buffer_length:])
        HR.is_warmed_up = True
        HR.update_loop()
        
    breathing_factor = 1.2
    
    timer_actuation1 = threading.Timer(0.1, breathing_biofeedback, ["1", B, breathing_factor, True] )
    timer_actuation1.start()
    
    timer_actuation2 = threading.Timer(0.1, breathing_biofeedback, ["2", B, breathing_factor, True] )
    #timer_actuation2 = threading.Timer(0.1, pulsating, ["2", True] )
    timer_actuation2.start()

In [168]:
def warmup_inflate(act_id, duration):
    global osc_client
    
    osc_client.send_message("/actuator/" + act_id + "/inflate", 100.0)
    time.sleep(duration)
    osc_client.send_message("/actuator/" + act_id + "/inflate", 0.0)


In [169]:
def deflate(act_id, duration):
    global osc_client
    
    osc_client.send_message("/actuator/" + act_id + "/inflate", -100.0)
    time.sleep(duration)
    osc_client.send_message("/actuator/" + act_id + "/inflate", 0.0)


### Different actuation sequences

In [205]:
# computes inhale and exhale interval duration as average duration of breath * breathing_factor 

def breathing_biofeedback(act_id, B, breathing_factor, inflate):
    global osc_client, actuation_flag
    
    if not actuation_flag:
        return
    
    if inflate:
        print("inhale: ", B.features['avg_inhale'])
        osc_client.send_message("/actuator/" + act_id + "/inflate", 100.0)
        timer_exhale = threading.Timer(B.features['avg_inhale'] * breathing_factor, breathing_biofeedback, [act_id, B, breathing_factor, False])
        timer_exhale.start()
        
        for i in range(1,10):
            osc_client.send_message("/actuator/" + act_id + "/inflate", 100.0)

    else:
        print("exhale: ", B.features['avg_inhale'])
        osc_client.send_message("/actuator/" + act_id + "/inflate", -100.0)
        timer_inhale = threading.Timer(B.features['avg_inhale'] * breathing_factor, breathing_biofeedback, [act_id, B, breathing_factor, True])
        timer_inhale.start()
        
        for i in range(1,10):
            osc_client.send_message("/actuator/" + act_id + "/inflate", -100.0)


In [14]:
# specify inhale and exhale interval duration in seconds

def breathing_intervals(act_id, inhale_duration, exhale_duration, inflate):
    global osc_client, actuation_flag
    
    if not actuation_flag:
        return
    
    if inflate:
        print("inhale: ", inhale_duration)
        osc_client.send_message("/actuator/" + act_id + "/inflate", 100.0)
        timer_exhale = threading.Timer(inhale_duration, breathing_intervals, [act_id, inhale_duration, exhale_duration, False])
        timer_exhale.start()

    else:
        print("exhale: ", exhale_duration)
        osc_client.send_message("/actuator/" + act_id + "/inflate", -100.0)
        timer_inhale = threading.Timer(exhale_duration, breathing_intervals, [act_id, inhale_duration, exhale_duration, True])
        timer_inhale.start()

In [176]:
def pulsating(act_id,inflate):
    global osc_client, actuation_flag
    
    if not actuation_flag:
        return
    
    if inflate:
        osc_client.send_message("/actuator/" + act_id + "/inflate", 50.0)
        timer_deflate = threading.Timer(3, pulsating, [act_id, False])
        timer_deflate.start()

    else:
        osc_client.send_message("/actuator/" + act_id + "/inflate", -50.0)
        timer_inflate = threading.Timer(3, pulsating, [act_id, True])
        timer_inflate.start()

In [16]:
# computes degree of inflation/deflation based on ECG features

# TODO: how to compute the inflation/deflation value?

def heartrate_actuation(act_id, HR, timer_interval):
    global osc_client, actuation_flag
    
    if not actuation_flag:
        return
    
    current_trend = HR.current_trends["hr_mean"]
    #current_trend = HR.current_trends["rmssd"]
    #current_trend = HR.current_trends["LF/HF ratio"]
    
    current_feature = HR.features[-1]["hr_mean"]
    #current_feature = HR.features[-1]["rmssd"]
    #current_feature = HR.features[-1]["LF/HF ratio"]
    
    if current_trend > 0:
        osc_client.send_message("/actuator/" + act_id + "/inflate", 100.0)
        #osc_client.send_message("/actuator/" + act_id + "/inflate", current_trend * 1000/current_feature)
        timer_exhale = threading.Timer(timer_interval, heartrate_actuation, [act_id, HR, timer_interval])
        timer_exhale.start()

    else:
        #osc_client.send_message("/actuator/inflate", current_trend * 1000/current_feature)
        osc_client.send_message("/actuator/" + act_id + "/inflate", -100.0)
        timer_inhale = threading.Timer(timer_interval, heartrate_actuation, [act_id, HR, timer_interval])
        timer_inhale.start()

#### Set up configuration: IP (BITalino, R-IoT or Node-RED), port and OSC address pattern

In [208]:
# Definitions
bitalino_ip = '192.168.0.100'
bitalino_port = 3333

riot_ip = "192.168.0.102"
riot_port = 31000 

# riot_ip = "192.168.1.40"
# riot_port = 12000 

#actuator_ip = '192.168.0.110'
actuator_ip = '192.168.0.100'
actuator_port = 32000

riot_address = "/raw/"
bitalino_address = "/0/bitalino"


# Configuration 
config_address = bitalino_address
config_ip = bitalino_ip
config_port = bitalino_port

osc_client = SimpleUDPClient(actuator_ip, actuator_port) 

actuation_flag = True

ecg_data = []
resp_data = []

B = biofeatures.breathing(data = np.ones(10), buffer_length=2000, srate=100)
HRV = biofeatures.hrv(data = np.ones(10), buffer_length=2000, srate=100)

my_dispatcher = dispatcher.Dispatcher()
my_dispatcher.map(config_address, data_handler, B, HRV)

server = osc_server.ThreadingOSCUDPServer((config_ip, config_port), my_dispatcher)
print("Serving on {}".format(server.server_address))

# Warmup
warmup_t = 10
timer_warmup = threading.Timer(warmup_t, warmup, [B,resp_data,HRV,ecg_data])
timer_warmup.start()

try: 
    server.serve_forever()
except KeyboardInterrupt:
    B.update_data_flag = False
    HRV.update_data_flag = False
    actuation_flag = False
    server.server_close()
    osc_client.send_message("/actuator/1/inflate", 0.0)
    osc_client.send_message("/actuator/2/inflate", 0.0)
except:
    raise

Serving on ('192.168.0.100', 3333)
Breathing WARMUP
{'breath_avg_len': 2.38, 'inhale_duration': 4.66, 'avg_inhale': 1.55, 'exhale_duration': 4.85, 'avg_exhale': 1.62, 'inhale_exhale_ratio': 0.96}
ECG WARMUP
[{'nni_mean': 502.35, 'hr_mean': 143.39, 'hr_std': 64.39, 'rmssd': 310.7, 'lf': 3298.04, 'hf': 8269.76, 'LF/HF ratio': 0.4}]
inhale:  1.55
inhale:  1.55
{'breath_avg_len': 2.5, 'inhale_duration': 5.15, 'avg_inhale': 1.72, 'exhale_duration': 4.83, 'avg_exhale': 1.61, 'inhale_exhale_ratio': 1.07}
[{'nni_mean': 502.35, 'hr_mean': 143.39, 'hr_std': 64.39, 'rmssd': 310.7, 'lf': 3298.04, 'hf': 8269.76, 'LF/HF ratio': 0.4}, {'nni_mean': 525.0, 'hr_mean': 135.73, 'hr_std': 63.21, 'rmssd': 295.73, 'lf': 1461.94, 'hf': 9526.86, 'LF/HF ratio': 0.15}]
{'breath_avg_len': 2.61, 'inhale_duration': 5.61, 'avg_inhale': 1.87, 'exhale_duration': 4.84, 'avg_exhale': 1.61, 'inhale_exhale_ratio': 1.16}
[{'nni_mean': 502.35, 'hr_mean': 143.39, 'hr_std': 64.39, 'rmssd': 310.7, 'lf': 3298.04, 'hf': 8269.76,

exhale:  2.05
exhale:  2.05
{'breath_avg_len': 3.15, 'inhale_duration': 8.06, 'avg_inhale': 2.02, 'exhale_duration': 7.69, 'avg_exhale': 1.54, 'inhale_exhale_ratio': 1.05}
[{'nni_mean': 488.52, 'hr_mean': 144.16, 'hr_std': 59.86, 'rmssd': 293.15, 'lf': 4958.35, 'hf': 5721.17, 'LF/HF ratio': 0.87}, {'nni_mean': 486.43, 'hr_mean': 143.99, 'hr_std': 58.75, 'rmssd': 288.08, 'lf': 5028.11, 'hf': 5625.74, 'LF/HF ratio': 0.89}, {'nni_mean': 491.38, 'hr_mean': 140.68, 'hr_std': 55.44, 'rmssd': 261.84, 'lf': 4528.28, 'hf': 6015.7, 'LF/HF ratio': 0.75}, {'nni_mean': 491.38, 'hr_mean': 140.68, 'hr_std': 55.44, 'rmssd': 261.84, 'lf': 4528.28, 'hf': 6015.7, 'LF/HF ratio': 0.75}, {'nni_mean': 491.38, 'hr_mean': 140.68, 'hr_std': 55.44, 'rmssd': 261.84, 'lf': 4528.28, 'hf': 6015.7, 'LF/HF ratio': 0.75}]
{'breath_avg_len': 2.71, 'inhale_duration': 8.93, 'avg_inhale': 1.79, 'exhale_duration': 7.31, 'avg_exhale': 1.46, 'inhale_exhale_ratio': 1.22}
[{'nni_mean': 486.43, 'hr_mean': 143.99, 'hr_std': 58.75

{'breath_avg_len': 2.85, 'inhale_duration': 9.64, 'avg_inhale': 1.61, 'exhale_duration': 10.34, 'avg_exhale': 1.48, 'inhale_exhale_ratio': 0.93}
[{'nni_mean': 449.27, 'hr_mean': 154.2, 'hr_std': 57.96, 'rmssd': 251.59, 'lf': 3243.41, 'hf': 6164.07, 'LF/HF ratio': 0.53}, {'nni_mean': -7.65, 'hr_mean': 149.2, 'hr_std': 61.98, 'rmssd': 3112.95, 'lf': 50024.19, 'hf': 20851.04, 'LF/HF ratio': 2.4}, {'nni_mean': 448.6, 'hr_mean': 154.86, 'hr_std': 58.76, 'rmssd': 256.58, 'lf': 2994.93, 'hf': 6255.11, 'LF/HF ratio': 0.48}, {'nni_mean': 447.91, 'hr_mean': 154.94, 'hr_std': 58.66, 'rmssd': 254.79, 'lf': 2742.3, 'hf': 6420.14, 'LF/HF ratio': 0.43}, {'nni_mean': -8.48, 'hr_mean': 136.46, 'hr_std': 57.69, 'rmssd': 3237.79, 'lf': 39040.68, 'hf': 27424.06, 'LF/HF ratio': 1.42}]
{'breath_avg_len': 2.85, 'inhale_duration': 9.42, 'avg_inhale': 1.57, 'exhale_duration': 10.56, 'avg_exhale': 1.51, 'inhale_exhale_ratio': 0.89}
[{'nni_mean': -7.65, 'hr_mean': 149.2, 'hr_std': 61.98, 'rmssd': 3112.95, 'lf': 

{'breath_avg_len': 2.85, 'inhale_duration': 9.81, 'avg_inhale': 1.4, 'exhale_duration': 10.17, 'avg_exhale': 1.7, 'inhale_exhale_ratio': 0.96}
[{'nni_mean': -10.89, 'hr_mean': 152.29, 'hr_std': 61.49, 'rmssd': 3057.77, 'lf': 35818.43, 'hf': 28250.39, 'LF/HF ratio': 1.27}, {'nni_mean': 436.82, 'hr_mean': 156.15, 'hr_std': 56.65, 'rmssd': 240.69, 'lf': 2393.61, 'hf': 5221.3, 'LF/HF ratio': 0.46}, {'nni_mean': -25.43, 'hr_mean': 153.98, 'hr_std': 60.82, 'rmssd': 3130.7, 'lf': 41798.84, 'hf': 26638.14, 'LF/HF ratio': 1.57}, {'nni_mean': 451.9, 'hr_mean': 154.49, 'hr_std': 58.68, 'rmssd': 280.15, 'lf': 2340.62, 'hf': 17468.72, 'LF/HF ratio': 0.13}, {'nni_mean': 435.58, 'hr_mean': 157.78, 'hr_std': 57.78, 'rmssd': 266.54, 'lf': 2147.98, 'hf': 4209.81, 'LF/HF ratio': 0.51}]
{'breath_avg_len': 2.85, 'inhale_duration': 9.9, 'avg_inhale': 1.65, 'exhale_duration': 10.08, 'avg_exhale': 1.68, 'inhale_exhale_ratio': 0.98}
[{'nni_mean': 436.82, 'hr_mean': 156.15, 'hr_std': 56.65, 'rmssd': 240.69, 'lf

In [210]:
from pythonosc import dispatcher, osc_server
from pythonosc.udp_client import SimpleUDPClient
import biofeatures

actuator_ip = '192.168.0.100'
actuator_port = 32000

osc_client = SimpleUDPClient(actuator_ip, actuator_port) 
osc_client.send_message("/actuator/1/inflate", -80.0)
osc_client.send_message("/actuator/2/inflate", -80.0)



In [209]:
deflate("1", 1)

In [191]:
warmup_inflate("1", 3)

In [177]:
actuation_flag = True

pulsating("1", True)

In [178]:
actuation_flag = False

In [167]:
osc_client.send_message("/actuator/1/inflate", 0.0)